In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import math

In [6]:
raw_dataset = pd.read_csv("/data/toy_dataset.csv")

In [7]:
raw_dataset.head(10)

,Number,City,Gender,Age,Income,Illness
0,1,Dallas,Male,41,40367.0,No
1,2,Dallas,Male,54,45084.0,No
2,3,Dallas,Male,42,52483.0,No
3,4,Dallas,Male,40,40941.0,No
4,5,Dallas,Male,46,50289.0,No
5,6,Dallas,Female,36,50786.0,No
6,7,Dallas,Female,32,33155.0,No
7,8,Dallas,Male,39,30914.0,No
8,9,Dallas,Male,51,68667.0,No
9,10,Dallas,Female,30,50082.0,No


In [8]:
raw_dataset.describe()

,Number,Age,Income
count,150000.000000,150000.000000,150000.000000
mean,75000.500000,44.950200,91252.798273
std,43301.414527,11.572486,24989.500948
min,1.000000,25.000000,-654.000000
25%,37500.750000,35.000000,80867.750000
50%,75000.500000,45.000000,93655.000000
75%,112500.250000,55.000000,104519.000000
max,150000.000000,65.000000,177157.000000


In [9]:
raw_dataset.dropna(inplace=True)
len(raw_dataset)

150000

In [10]:
raw_dataset.drop(columns=['Number'], inplace=True)
raw_dataset.columns

Index([&#39;City&#39;, &#39;Gender&#39;, &#39;Age&#39;, &#39;Income&#39;, &#39;Illness&#39;], dtype=&#39;object&#39;)

In [11]:
# categorization

def discretization(attr_list):

    sorted_list = sorted(attr_list)

    '''
    Our Discretization Method:

    Q 0            Q 1            Q 2            Q 3             Q 4
    0 % --------- 25 % --------- 50 % --------- 75 % --------- 100 % 
    
    score - 0             1               2              3

    '''

    attr_map = dict()
    multiple = len(attr_list) / 4
    next = multiple

    for index in range(4):
        attr_map[index] = sorted_list[int(next-1)] 
        next += multiple
    result = []

    for val in attr_list:
        for i in range(4):
            if val <= attr_map[i]:
                result.append(i)
                break
    
    return result


def categoric_encoder(attr_list, attr_map = {}):

    i = 0

    if len(attr_map) == 0:
        attr_set = set(attr_list)
        for a in attr_set:
            attr_map[a] = i
            i += 1
        
    result = []
    
    for a in attr_list:
        try:
            result.append(attr_map[a])
        except:
            print("'" + a + "'")
    
    return result

In [12]:
'''
Index(['City', 'Gender', 'Age', 'Income', 'Illness'], dtype='object')
We need to discretize age & income as they are continuous attribute 
encode rest of the categorical attributes
'''
dataset = raw_dataset.copy()

dataset['City'] = categoric_encoder(dataset['City'])
dataset['Gender'] = categoric_encoder(dataset['Gender'], {'Female': 0, 'Male': 1})
dataset['Age'] = discretization(dataset['Age'])
dataset['Income'] = discretization(dataset['Income'])
dataset['Illness'] = categoric_encoder(dataset['Illness'], {'Yes': 1, 'No': 0})

In [13]:
dataset.head()

,City,Gender,Age,Income,Illness
0,4,1,1,0,0
1,4,1,2,0,0
2,4,1,1,0,0
3,4,1,1,0,0
4,4,1,2,0,0


In [14]:
dataset.describe()

,City,Gender,Age,Income,Illness
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000
mean,3.713187,0.558667,1.457700,1.499953,0.080927
std,2.184899,0.496548,1.116975,1.118020,0.272723
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.750000,0.000000
50%,4.000000,1.000000,1.000000,1.000000,0.000000
75%,6.000000,1.000000,2.000000,2.000000,0.000000
max,7.000000,1.000000,3.000000,3.000000,1.000000


## Decision Tree Classifier

In [19]:

class Node:

    def __init__(self, attribute = None):
        self.attribute = attribute
        self.children = {}
        self.group = None


class DecisionTreeClassifier:

    def __init__(self):
        self.attr_set = []
        self.class_set = {}
        self.root = None
        self.num = 0

    def set_metadata(self, X, Y):

        for i in range(self.num): 
            # accessing the attribute columns one at a time

            x = X[:, i].flatten()
            y = Y.flatten()

            # preparing details of ith attribute column
            attr_details = {}

            # dictionary for different values of categorical attribute column
            for a in set(x):
                attr_details[a] = set()

            for i, a in enumerate(x):
                attr_details[a].add(i)
            
            self.attr_set.append(attr_details)
        
        # preparing details of objective class
        for c in set(y):
            self.class_set[c] = set()
        
        for i, c in enumerate(y):
            self.class_set[c].add(i)

        return


    def entropy(self, p):

        '''
        Entropy = - P X Log(P)
        '''
        if p == 0:
            return 0
        return (-1 * p * math.log2(p))


    def get_base_entropy(self, remainder_indices):

        result = 0
        if (len(remainder_indices) == 0): return result

        for c in self.class_set.keys():
            p = len(self.class_set[c].intersection(remainder_indices)) / len(remainder_indices)
            result += self.entropy(p)
        
        return result


    def get_group(self, remainder_indices):
        
        best_class = None
        best_score = 0

        for c in self.class_set.keys():

            score = len( self.class_set[c].intersection(remainder_indices) )

            if (score > best_score):
                best_class = c
                best_score = score
            
            return best_class
    

    def find_best_attribute(self, attributes, indices, base):

        best_attribute = None
        best_gain = 0

        for a in list(attributes): # attribute
            score = 0
            for c in self.attr_set[a].keys(): # subclasses in every attribute

                new_indices = self.attr_set[a][c].intersection(indices)
                
                p = len(new_indices) / len(indices)

                entropy = self.get_base_entropy(new_indices)

                score += p * entropy
            
            gain = base - score

            if (gain > best_gain):
                best_gain = gain
                best_attribute = a
        
        return best_attribute


    def make_tree(self, remainder_indices, remainder_attributes, depth):
        
        if len(remainder_indices) == 0:

            root = Node()
            root.group = list(self.class_set.keys())[0]
            return root

        base_entropy = self.get_base_entropy(remainder_indices)

        if depth == 0 or len(remainder_attributes) == 0 or base_entropy == 0:

            root = Node()

            root.group = self.get_group(remainder_indices)

            return root

        best_attribute = self.find_best_attribute(remainder_attributes, remainder_indices, base_entropy)
        
        remainder_attributes.remove(best_attribute)
        
        root = Node(best_attribute)

        for c in self.attr_set[best_attribute].keys(): 

            new_indices = self.attr_set[best_attribute][c].intersection(remainder_indices)
            
            root.children[c] = self.make_tree(new_indices, remainder_attributes, depth-1)

        return root


    def fit(self, X, Y, depth):
        
        self.num = X.shape[1]
        self.set_metadata(X, Y)

        remainder_indices = set([i for i in range(Y.shape[0])])
        remainder_attributes = set([i for i in range(self.num)])

        self.root = self.make_tree(remainder_indices, remainder_attributes, depth)

        return


    def predict(self, x):

        temp = self.root
        while(temp.attribute != None):

            c = x[temp.attribute]
            temp = temp.children[c]

        return temp.group


    def evaluate(self, X, Y):

        score = 0
        total = X.shape[0]

        for i, x in enumerate(X):

            c = self.predict(x)

            if (c == Y[i]):
                score += 1
        
        return (score/total)


In [20]:
data = dataset.to_numpy()
X = data[:, :-1]
Y = data[:,  -1]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [21]:
model = DecisionTreeClassifier()
model.fit(X_train, Y_train, 10)

In [22]:
'''
Evaluating on the data it is trained
'''
model.evaluate(X_train, Y_train)

0.91865

In [23]:
'''
Evaluating on testing data
'''
model.evaluate(X_test, Y_test)

0.9207666666666666